In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score)
from utils.data import *
from utils.deep_learning import *
from utils.measuring_performance import *
from utils.misc import *
sys.path.append('../Fork/DeepCTR')
from deepctr.inputs import DenseFeat, SparseFeat, get_feature_names
from deepctr.inputs import input_from_feature_columns, get_linear_logit, build_input_features, combined_dnn_input, DEFAULT_GROUP_NAME
from deepctr.layers.core import PredictionLayer, DNN
from deepctr.layers.interaction import FM
from deepctr.layers.utils import concat_func, add_func

In [4]:
DATA_DIR = os.path.abspath('../../Data/display_advertising_challenge/processed')
MODEL_DIR = os.path.abspath('models')
USE_TFRECORD = True

In [5]:
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

In [6]:
train_dataset_type = 'train+valid'
test_dataset_type = 'test'
model_type = 'deepfm'
model_path = os.path.join(MODEL_DIR, '_'.join([model_type, 'model', train_dataset_type]))

In [7]:
target_name, num_feature_names, cat_feature_names, n_categories = load_pickle(
    os.path.join(DATA_DIR, '_'.join([train_dataset_type, 'metadata.pkl'])))

In [8]:
if USE_TFRECORD:
    train_dataset_path = os.path.join(DATA_DIR, '_'.join(['dataset', train_dataset_type]) + '.tfrecord')
    test_dataset_path = os.path.join(DATA_DIR, '_'.join(['dataset', test_dataset_type]) + '.tfrecord')
    
    shuffle_buffer_size = 2 ** 20
    train_dataset = extract_dataset(train_dataset_path, compression_type='GZIP', 
                                    shuffle_buffer_size=shuffle_buffer_size, is_training=True)
    test_dataset = extract_dataset(test_dataset_path, compression_type='GZIP', 
                                    shuffle_buffer_size=shuffle_buffer_size, is_training=True)
    
    n, m = 36672494, 9168123
    # n = get_n_examples(train_dataset)
    # m = get_n_examples(test_dataset)
    
else:
    df_y_train = pd.read_pickle(os.path.join(DATA_DIR, '_'.join(['df', 'y', train_dataset_type]) + '.pkl'))
    df_X_train = pd.read_pickle(os.path.join(DATA_DIR, '_'.join(['df', 'X', train_dataset_type]) + '.pkl'))
    df_y_test = pd.read_pickle(os.path.join(DATA_DIR, '_'.join(['df', 'y', test_dataset_type]) + '.pkl'))
    df_X_test = pd.read_pickle(os.path.join(DATA_DIR, '_'.join(['df', 'X', test_dataset_type]) + '.pkl'))

    train_model_input = {column: df_X_train[column].values for column in df_X_train.columns}
    test_model_input = {column: df_X_test[column].values for column in df_X_test.columns}
    
    n = df_y_train.shape[0]
    m = df_y_test.shape[0]

In [11]:
embedding_dim = 4
num_features = [DenseFeat(feature, 1) for feature in num_feature_names]
cat_features = [SparseFeat(feature, vocabulary_size=n_categories[feature], 
                           embedding_dim=embedding_dim, use_hash=False) for feature in cat_feature_names]
linear_features = num_features + cat_features
dnn_features = num_features + cat_features
all_feature_names = get_feature_names(num_features + cat_features)

In [14]:
features = build_input_features(
    linear_features + dnn_features)

In [16]:
inputs_list = list(features.values())

In [18]:
features

OrderedDict([('I1', <tf.Tensor 'I1_2:0' shape=(None, 1) dtype=float32>),
             ('I2', <tf.Tensor 'I2_2:0' shape=(None, 1) dtype=float32>),
             ('I3', <tf.Tensor 'I3_2:0' shape=(None, 1) dtype=float32>),
             ('I4', <tf.Tensor 'I4_2:0' shape=(None, 1) dtype=float32>),
             ('I5', <tf.Tensor 'I5_2:0' shape=(None, 1) dtype=float32>),
             ('I6', <tf.Tensor 'I6_2:0' shape=(None, 1) dtype=float32>),
             ('I7', <tf.Tensor 'I7_2:0' shape=(None, 1) dtype=float32>),
             ('I8', <tf.Tensor 'I8_2:0' shape=(None, 1) dtype=float32>),
             ('I9', <tf.Tensor 'I9_2:0' shape=(None, 1) dtype=float32>),
             ('I10', <tf.Tensor 'I10_2:0' shape=(None, 1) dtype=float32>),
             ('I11', <tf.Tensor 'I11_2:0' shape=(None, 1) dtype=float32>),
             ('I12', <tf.Tensor 'I12_2:0' shape=(None, 1) dtype=float32>),
             ('I13', <tf.Tensor 'I13_2:0' shape=(None, 1) dtype=float32>),
             ('C1', <tf.Tensor 'C1_2:0' sha

In [ ]:
model = DeepFM(linear_features, dnn_features, task='binary')
if len(get_available_gpus()) > 1:
    model = tf.keras.utils.multi_gpu_model(model, gpus=n_gpus)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC()])

In [ ]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=1, mode='max'),
             tf.keras.callbacks.ModelCheckpoint(
                 filepath=model_path + '.h5', monitor='val_auc', save_best_only=True)]

In [ ]:
epochs = 10
batch_size = 2 ** 17

In [ ]:
if USE_TFRECORD:
    steps_per_epoch = n // batch_size
    validation_steps = m // batch_size
    
    train_dataset = transform_dataset(train_dataset, num_feature_names, cat_feature_names, target_name)
    test_dataset = transform_dataset(test_dataset, num_feature_names, cat_feature_names, target_name)
    
    train_generator = load_dataset(train_dataset, batch_size=batch_size, is_training=False)
    valid_generator = load_dataset(test_dataset, batch_size=batch_size, is_training=False)
        
    else:
        history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=epochs, 
                                      verbose=True, validation_data=valid_generator, 
                                      validation_steps=validation_steps, callbacks=callbacks)
    
else:
    history = model.fit(train_model_input, df_y_train.values, batch_size=batch_size, epochs=epochs, 
                        verbose=True, validation_data=(test_model_input, df_y_test.values), callbacks=callbacks)

In [ ]:
model.save(model_path + '.h5')
dump_pickle(model_path + '_history.pkl', history.history)

In [ ]:
model = tf.keras.models.load_model(model_path + '.h5', custom_objects=custom_objects)

In [ ]:
if USE_TFRECORD:
    test_dataset = extract_dataset(test_dataset_path, compression_type='GZIP', 
                                   shuffle_buffer_size=shuffle_buffer_size, is_training=False)
    test_dataset = transform_dataset(test_dataset, num_feature_names, cat_feature_names, target_name) 
    y_true = get_target(test_dataset)
    
    test_generator = load_dataset(test_dataset, batch_size=m, is_training=False)
    y_score = model.predict_generator(test_generator).ravel()
    
else:
    y_true = df_y_test.values
    y_score = model.predict(test_model_input).ravel()

In [ ]:
ctr = y_true.mean()
y_pred = get_y_pred(y_score, threshold=ctr)

norm_entropy = get_norm_entropy(y_true, y_score)
calibration = y_score.mean() / ctr
accuracy, precision, recall, f1 = accuracy_score(y_true, y_pred), precision_score(y_true, y_pred), \
    recall_score(y_true, y_pred), f1_score(y_true, y_pred)

confusion_matrix = plot_confusion_matrix(y_true, y_pred)
auroc = plot_roc_curve(y_true, y_score)
auprc = plot_pr_curve(y_true, y_score)
_ = plot_lift_curve(y_true, y_score)
_ = plot_class_density(y_true, y_score, threshold=ctr)

In [ ]:
dump_pickle(os.path.join(MODEL_DIR, '_'.join([model_type, 'metric', train_dataset_type]) + '.pkl'), 
            (norm_entropy, calibration, accuracy, precision, recall, f1, confusion_matrix, auroc, auprc))

In [ ]:
test_dataset_type = 'quiz'
score_path = os.path.join(MODEL_DIR, '_'.join([model_type, 'score', test_dataset_type]) + '.pkl')

In [ ]:
if USE_TFRECORD:
    test_dataset_path = os.path.join(DATA_DIR, '_'.join([model_type, 'dataset', test_dataset_type]) + '.tfrecord')
    test_dataset = extract_dataset(test_dataset_path, compression_type='GZIP', 
                                    shuffle_buffer_size=shuffle_buffer_size, is_training=False)
    
    m = 6042135 
    # m = get_n_examples(test_dataset)
    
    test_dataset = transform_dataset(test_dataset, num_feature_names, cat_feature_names, target_name) 
    test_generator = load_dataset(test_dataset, batch_size=m, is_training=False)
    y_score = model.predict_generator(test_generator).ravel()
    
else:
    df_X_test = pd.read_pickle(os.path.join(DATA_DIR, '_'.join(['df', 'X', test_dataset_type]) + '.pkl'))
    test_model_input = {column: df_X_test[column].values for column in df_X_test.columns}
    y_score = model.predict(test_model_input).ravel()

In [ ]:
dump_pickle(score_path, y_score)